In [1]:
# from abc import ABC, abstractmethod
# from typing import Any, dict, Optional, Tuple


# class BaseFormatter(ABC):
#     def __init__(self, indent_size: int):
#         self._space: str = " "
#         self._sep: str = "|"
#         self._comment: str = "#"
#         self._req: str = "Required:"
#         self._desc: str = "Description:"
#         self._type: str = "Type:"
#         self._indent_size = indent_size

#     @abstractmethod
#     def __call__(self, properties: dict[str, Any], _level: int = 0) -> str:
#         ...

#     def _get_indent(self, _level: int) -> str:
#         return self._space * (self._indent_size * _level)

#     def _extract_metadata(self, property_content: dict[str, Any]) -> Tuple[Any, Any, bool, str, dict[str, Any], dict[str, Any]]:
#         default: Optional[Any] = property_content.pop("default", None)
#         value_type: Optional[str] = property_content.pop("type", None)
#         required: bool = property_content.pop("required", False)
#         description: str = property_content.pop("description", "")

#         nested, additional = {}, {}
#         for k, v in property_content.items():
#             if k == "title": # TODO refactor
#                 continue

#             if isinstance(v, dict) and any(isinstance(val, (dict, list)) for val in v.values()):
#                 nested[k] = v
#             else:
#                 additional[k] = v

#         return default, value_type, required, description, additional, nested


# class YAMLFormatter(BaseFormatter):
#     def __init__(self):
#         super().__init__(indent_size=2)

#     def __call__(self, properties: dict[str, Any], _level: int = 0) -> str:
#         lines: list[str] = []
#         indent: str = self._get_indent(_level)

#         for name, property_content in properties.items():
#             if not isinstance(property_content, dict):
#                 continue

#             default, value_type, required, description, additional, nested = self._extract_metadata(property_content)
#             line: str = self._format_line(name, default, value_type, required, description, additional, indent)
#             lines.append(line)

#             if nested:
#                 lines.append(self(nested, _level + 1))

#         return "\n".join(lines)

#     def _format_line(
#         self,
#         name: str,
#         default: Any,
#         value_type: Any,
#         required: bool,
#         description: str,
#         additional: dict[str, Any],
#         indent: str,
#     ) -> str:
#         additional_info = ", ".join(f"{k}: {v}" for k, v in additional.items())

#         line = (
#             # f"{(indent + self._comment + ' CLASS') if value_type is None else ''}\n"
#             f"{indent}{name}:"
#             f"{self._space if default is None else f' {default} '}"
#             f"{self._comment} "
#             f"{(self._type + self._space + str(value_type) + self._space + self._sep + self._space) if value_type else ''}"
#             f"{(additional_info + self._space + self._sep + self._space) if additional_info else ''}"
#             f"{self._req} {required} {self._sep} {self._desc} {description}"
#         )
#         return line


In [2]:
from pydantic import BaseModel, Field


class CommonSettings(BaseModel):
    name: str = Field(..., min_length=3, max_length=50, description="name in CommonSettings")
    enabled: bool = Field(default=True, description="enabled in CommonSettings")
    priority: int = Field(default=1, ge=1, le=10, description="priority in CommonSettings")


class InnerLayer1(BaseModel):
    commonsettings: CommonSettings = Field(..., description="commongsettings in InnerLayer1")
    random: bool = Field(..., description="just random")


class InnerLayer2(BaseModel):
    inner_layer_1: InnerLayer1 = Field(..., description="inner1 property Innerlayer2")


class InnerLayer3(BaseModel):
    inner_layer_2: InnerLayer2 = Field(..., description="inner2 in Innerlayer3")


class InnerLayer4(BaseModel):
    inner_layer_3: InnerLayer3 = Field(..., description="inner3 in Innerlayer4")


class AppConfig(BaseModel):
    name: str = Field(..., min_length=3, max_length=50, description="name in Appconfig")
    inner_layer_4: InnerLayer4 = Field(..., description="inner4 in appfoncig", title="öaldkjf")
    inner_layer_2: InnerLayer2 = Field(..., description="inner2 in inner2")


# Main

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from typing import Any

from confflow.common.utils import resolve_schemas, resolve_properties, resolve

In [5]:
model_json_schema: dict[str, Any] = AppConfig.model_json_schema()

In [6]:
schema = resolve(model_json_schema)

In [7]:
schema

{'name': {'description': 'name in Appconfig',
  'maxLength': 50,
  'minLength': 3,
  'title': 'Name',
  'type': 'string',
  'required': True},
 'inner_layer_4': {'properties': {'inner_layer_3': {'properties': {'inner_layer_2': {'properties': {'inner_layer_1': {'properties': {'commonsettings': {'properties': {'name': {'description': 'name in CommonSettings',
            'maxLength': 50,
            'minLength': 3,
            'title': 'Name',
            'type': 'string',
            'required': False},
           'enabled': {'default': True,
            'description': 'enabled in CommonSettings',
            'title': 'Enabled',
            'type': 'boolean',
            'required': False},
           'priority': {'default': 1,
            'description': 'priority in CommonSettings',
            'maximum': 10,
            'minimum': 1,
            'title': 'Priority',
            'type': 'integer',
            'required': False}},
          'required': False,
          'title': 'CommonS

In [17]:
app_config = AppConfig(
    name="AAAAAAAAA",
    inner_layer_4=InnerLayer4(
        inner_layer_3=InnerLayer3(
            inner_layer_2=InnerLayer2(
                inner_layer_1=InnerLayer1(
                    commonsettings=CommonSettings(
                        name="BBBBBBBBBBBBB",
                        enabled=True,
                        priority=10
                    ),
                    random=True
                )
            )
        )
    ),
    inner_layer_2=InnerLayer2(
        inner_layer_1=InnerLayer1(
            commonsettings=CommonSettings(
                name="CCCCCCCCCCCC",
                enabled=False,
                priority=10
            ),
            random=False
        )
    )
)

In [18]:
app_config.model_dump() | model_json_schema

{'name': 'AAAAAAAAA',
 'inner_layer_4': {'inner_layer_3': {'inner_layer_2': {'inner_layer_1': {'commonsettings': {'name': 'BBBBBBBBBBBBB',
      'enabled': True,
      'priority': 10},
     'random': True}}}},
 'inner_layer_2': {'inner_layer_1': {'commonsettings': {'name': 'CCCCCCCCCCCC',
    'enabled': False,
    'priority': 10},
   'random': False}},
 '$defs': {'CommonSettings': {'properties': {'name': {'description': 'name in CommonSettings',
     'maxLength': 50,
     'minLength': 3,
     'title': 'Name',
     'type': 'string'},
    'enabled': {'default': True,
     'description': 'enabled in CommonSettings',
     'title': 'Enabled',
     'type': 'boolean'},
    'priority': {'default': 1,
     'description': 'priority in CommonSettings',
     'maximum': 10,
     'minimum': 1,
     'title': 'Priority',
     'type': 'integer'}},
   'required': ['name'],
   'title': 'CommonSettings',
   'type': 'object'},
  'InnerLayer1': {'properties': {'commonsettings': {'$ref': '#/$defs/CommonSetti